In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
pd.options.display.max_columns = 100
train = pd.read_csv("/kaggle/input/dkdacon/train.csv")
test = pd.read_csv("/kaggle/input/dkdacon/test.csv")

In [ ]:
# 이상치 제거 1 - 분포
# train = train.drop(train[train["u"] < -10].index) # 애매한데
# train = train.drop(train[(train["g"] < -15000)].index) 
# train = train.drop(train[train["r"] > 60].index)
# train = train.drop(train[train["i"] < -20000].index)
# train = train.drop(train[train["z"] < -6000].index)

# plt.figure(figsize=(20,12))
# sns.boxplot(train["class"], train["u"])
# sns.countplot(alldata["nObserve"], alldata["class"])


# # 이상치 제거 2 - 상관도
# train = train.drop(train[(train["g"] > 80) | (train["g"] < -40)].index)
# train = train.drop(train[train["redshift"] < -100].index)
# train = train.drop(train[train["z"] > 6000].index)
# train = train.drop(train[(train["redshift"] < -40) & (train["nObserve"] > 5)].index)
# train
# plt.figure(figsize=(20,12))
# sns.scatterplot(alldata["i"], alldata["z"])
# train

alldata = pd.concat([train,test])
alldata

In [ ]:
# from sklearn.decomposition import PCA

# pca = PCA(n_components = 1)
# alldata["new1"] = pca.fit_transform(alldata[["u","g","r","i","z"]])
# alldata

In [ ]:
alldata["u-g"] = alldata["u"]-alldata["g"]
alldata["g-r"] = alldata["g"]-alldata["r"]
alldata["r-i"] = alldata["r"]-alldata["i"]
alldata["i-z"] = alldata["i"]-alldata["z"]

alldata["d1"] = alldata["dered_u"]-alldata["dered_g"]
alldata["d2"] = alldata["dered_g"]-alldata["dered_r"]
alldata["d3"] = alldata["dered_r"]-alldata["dered_i"]
alldata["d4"] = alldata["dered_i"]-alldata["dered_z"]

alldata["minusOD"] = alldata["nObserve"] - alldata["nDetect"]
alldata["plusOD"] = alldata["nObserve"] + alldata["nDetect"]

# alldata["diff_u"] = alldata["u"] - alldata["dered_u"]
# alldata["diff_g"] = alldata["g"] - alldata["dered_g"]
# alldata["diff_r"] = alldata["r"] - alldata["dered_r"]
# alldata["diff_i"] = alldata["i"] - alldata["dered_i"]
# alldata["diff_z"] = alldata["z"] - alldata["dered_z"]

# alldata["new1"] = alldata["redshift"] * alldata["new1"]

# alldata["new1"] = alldata["redshift"] / alldata["u"]
# alldata["new2"] = alldata["redshift"] / alldata["g"]

# alldata["new1"] = alldata["redshift"] * alldata["u-g"]
# alldata["new2"] = alldata["redshift"] * alldata["g-r"]
# alldata["new3"] = alldata["redshift"] * alldata["r-i"]
# alldata["new4"] = alldata["redshift"] * alldata["i-z"]

 

alldata

In [ ]:
alldata2 = alldata.drop(["id","class"], axis=1)
alldata2 = alldata2.drop(["airmass_u","airmass_g","airmass_r","airmass_i","airmass_z"], axis=1 )
# alldata2 = alldata2.drop(["nDetect","nObserve"], axis=1 )

alldata2

In [ ]:
train2 = alldata2[:len(train)]
test2 = alldata2[len(train):]

In [ ]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
x_train,x_valid, y_train, y_valid = train_test_split(train2, train["class"], test_size = 0.2, random_state = 42, shuffle = True)

hr2_accuracy_list = []

max_depth_list = [5,6,7,8,9,10]
subsample_list = [0.6,0.7,0.8,0.9,1.0]

def hr2_grid_search(max_depth, subsample): 
    xgb_model = XGBClassifier(n_estimators = 200, max_depth=max_depth ,subsample=subsample ,tree_method = "gpu_hist", n_jobs = -1)
    xgb_pred = xgb_model.fit(x_train, y_train).predict(x_valid)
    return([max_depth, subsample, accuracy_score(y_valid, xgb_pred)])
      

for max_depth in max_depth_list:
    for subsample in subsample_list:
        hr2_accuracy_list.append(hr2_grid_search(max_depth, subsample))
    

hr2_xgb_df = pd.DataFrame(hr2_accuracy_list, columns=['tree depth', 'subsample rate', 'accuracy'])
print(hr2_xgb_df["accuracy"].argmax())
hr2_xgb_df

In [ ]:
from sklearn.model_selection import StratifiedKFold,train_test_split
from catboost import CatBoostClassifier
from xgboost import XGBClassifier

# 기본 평가셋 
x_train,x_valid, y_train, y_valid = train_test_split(train2, train["class"], test_size = 0.2, random_state = 42, shuffle = True)

sfk = StratifiedKFold(n_splits = 10, random_state = 42, shuffle = True)
result = 0


# catboost모델
# ca = CatBoostClassifier(n_jobs = -1, n_estimators = 15000, task_type = "GPU", max_depth = 7, learning_rate = 0.02)
# ca.fit(x_train,y_train, eval_set = (x_valid, y_valid), early_stopping_rounds = 30)

# 앙상블용1 0.6545
# xgb = XGBClassifier(n_estimators = 1000, tree_method = "gpu_hist", n_jobs = -1, max_depth = 10,  colsample_bytree = 0.7)
# xgb.fit(x_train, y_train, eval_set = [(x_valid, y_valid)], early_stopping_rounds = 50)

# 앙상블용2 0.6545
# xgb = XGBClassifier(n_estimators = 1000, tree_method = "gpu_hist", n_jobs = -1, max_depth = 8,  colsample_bytree = 0.6)
# xgb.fit(x_train, y_train, eval_set = [(x_valid, y_valid)], early_stopping_rounds = 50)


# 찐 XGB모델
xgb = XGBClassifier(n_estimators = 1000, tree_method = "gpu_hist", n_jobs = -1, max_depth = 9)
xgb.fit(x_train, y_train, eval_set = [(x_valid, y_valid)], early_stopping_rounds = 50)

# 제출용
# for i,j in sfk.split(train2,train["class"]):
#     x_train = train2.iloc[i]
#     x_valid = train2.iloc[j]
#     y_train = train["class"].iloc[i]
#     y_valid = train["class"].iloc[j]

#     xgb = XGBClassifier(n_estimators = 1000, tree_method = "gpu_hist", n_jobs = -1, max_depth = 9)
#     xgb.fit(x_train, y_train, eval_set = [(x_valid, y_valid)], early_stopping_rounds = 50)
#     result += xgb.predict_proba(test2)/10

# merror: Multiclass classification error rate

# 0.06484

# 0.06631 7
# 0.06556 8

# col = 0.7- 0.06536 
# 0.06553 9         0.06534     0.06556

# 0.06636 10


In [ ]:
sub = pd.read_csv("/kaggle/input/dkdacon/sample_submission.csv")
sub.head()

In [ ]:
sub["class"] = result.argmax(1)
sub

In [ ]:
sub.to_csv("dk_dacon.csv", index=False)

In [ ]:
pd.Series(xgb.feature_importances_, index = train2.columns).sort_values(ascending = False).reset_index()